In [20]:
import pandas as pd
import os

# for category mapping
COLUMN_MAP = {
    "date": ["date", "Date", "timestamp", "Datetime"],
    "open": ["open", "Open", "OPEN", "Open Price"],
    "high": ["high", "High", "HIGH"],
    "low": ["low", "Low", "LOW"],
    "close": ["Close", "close", "CLOSE","Close/Last","CLOSE/LAST","close/last"],
    "adj_close": ["Adj Close", "adj_close", "adj close"],
    "volume": ["volume", "Volume", "VOL", "vol"]
}

def standardize_columns(df):
    """Rename CSV columns to standard names."""
    new_cols = {}
    for std_col, aliases in COLUMN_MAP.items():
        for alias in aliases:
            if alias in df.columns:
                new_cols[alias] = std_col
                break
    df = df.rename(columns=new_cols)
    return df

def load_and_clean_csv(file_path):
    """Load and clean a single CSV file."""
    df = pd.read_csv(file_path)
    df = standardize_columns(df)

    required = ["date", "open", "high", "low", "close", "volume"]
    for col in required:
        if col not in df.columns:
            df[col] = None  # create missing column with NaN values
    
    # drop rows if critical columns are missing
    df = df.dropna(subset=["date", "close"])

    # converts to a specific format: prices
    numeric_cols = ["open", "high", "low", "close", "adj_close", "volume"]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].replace('[\$,]', '', regex=True)  # remove $ and ,
            df[col] = pd.to_numeric(df[col], errors="coerce")  # convert to float

    # converts to a specific format: dates
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"])
    df = df.sort_values(by="date").drop_duplicates(subset=["date"])

    return df


if __name__ == "__main__":
    file_path = "ms-hd.csv"  # Change this to your CSV file
    cleaned_df = load_and_clean_csv(file_path)

    print("CLEANED DATASET PREVIEW:")
    print(cleaned_df.head(10))

    # Create "clean" folder if it doesn't exist
    output_folder = "clean"
    os.makedirs(output_folder, exist_ok=True)

    # Add "clean_" prefix to filename
    original_name = os.path.basename(file_path) 
    clean_name = "clean_" + original_name
    output_path = os.path.join(output_folder, clean_name)

    if not os.path.exists(output_path):
        cleaned_df.to_csv(output_path, index=False)
        print(f"Saved cleaned file to: {output_path}")
    else:
        print(f"File already exists: {output_path}")



✅ Cleaned dataset preview:
           date  close    volume   open   high     low
2514 2015-08-10  47.33  23045530  46.95  47.49  46.840
2513 2015-08-11  46.41  28757610  46.82  46.94  45.900
2512 2015-08-12  46.74  30184710  46.19  46.90  45.705
2511 2015-08-13  46.73  22612650  47.06  47.10  46.490
2510 2015-08-14  47.00  21460830  46.53  47.10  46.520
2509 2015-08-17  47.32  21039660  46.81  47.45  46.570
2508 2015-08-18  47.27  23572990  46.84  47.43  46.700
2507 2015-08-19  46.61  31472060  46.78  47.08  46.300
2506 2015-08-20  45.66  36176380  46.07  46.47  45.660
2505 2015-08-21  43.07  69463900  45.30  45.48  43.070
⚠️ File already exists: clean\clean_ms-hd.csv


<>:42: SyntaxWarning: invalid escape sequence '\$'
<>:42: SyntaxWarning: invalid escape sequence '\$'
D:\CONFIG\TEMP\ipykernel_16944\1289574252.py:42: SyntaxWarning: invalid escape sequence '\$'
  df[col] = df[col].replace('[\$,]', '', regex=True)  # remove $ and ,


In [17]:
cleaned_df

,date,close,volume,open,high,low
2514,2015-08-10,29.9300,219404040,29.1325,29.9975,29.1325
2513,2015-08-11,28.3725,387282680,29.4525,29.5450,28.3325
2512,2015-08-12,28.8100,406373600,28.1325,28.8550,27.4075
2511,2015-08-13,28.7875,193630560,29.0100,29.1000,28.6350
2510,2015-08-14,28.9900,171505480,28.5800,29.0775,28.5025
...,...,...,...,...,...,...
4,2025-08-04,203.3500,75109300,204.5050,207.8800,201.6750
3,2025-08-05,202.9200,44155080,203.4000,205.3400,202.1600
2,2025-08-06,213.2500,108483100,205.6300,215.3800,205.5900
1,2025-08-07,220.0300,90224830,218.8750,220.8500,216.5800
